# IMPORTING LIBRARIES FOR THE SCRAPPING

In [ ]:
SELENIUM HAS TO BE INSTALLED LIKEWISE CHROME WEB DRIVER AND ADDED TO THE RIGHT PATH TO RUN THIS SCRIPT

In [226]:
#pip install selenium --quiet
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# INSTANTIATING THE TOOL FOR THE SCRAPPING

In [216]:
chrome_driver_path = "C:\\Users\\dell\\Documents\\chrome driver\\chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.get("https://www.marchespublics.gov.ma/index.php?page=entreprise.EntrepriseAdvancedSearch&AllCons&EnCours"
           "&searchAnnCons")

C:\Users\dell\AppData\Local\Temp\ipykernel_5792\2573727559.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


# EXTRACTION SCRIPT FOR MOROCCO

In [217]:
##CREATING LIST TO HOLD EACH SCRAPPED ITEM

morocco_dataset = pd.DataFrame()
ref_texts = []
city_texts = []
category_texts = []
object_texts = []
public_purchase_texts = []
datetime_end_list = []
start_dt_list = []

for i in range(170):
    
    #SCRAPPING THE REFERENCE IDS
    
    # refs=driver.find_elements_by_class_name("ref")
    refs = driver.find_elements(by=By.CLASS_NAME, value="ref")
    for ref in refs:
        
        ref_text = ref.text
        ref_texts.append(ref_text)
        
    charToDelete = ''
    ref_text2 = []
    for character in ref_texts:
        if character == charToDelete:
            continue
        ref_text2.append(character)
   
        
        #SCRAPPING THE LOCATION FOR THE CONTRACTS
    for i in range(1, 11):
        cities = driver.find_element(by=By.ID,
                                     value=f"ctl0_CONTENU_PAGE_resultSearch_tableauResultSearch_ctl{i}_panelBlocLieuxExec")
        city_texts.append(cities.text)
        
        #SCRAPPING THE PRODUCT CATEGORY SECTION
    for i in range(1, 11):
        category = driver.find_element(by=By.ID,
                                       value=f"ctl0_CONTENU_PAGE_resultSearch_tableauResultSearch_ctl{i}_panelBlocCategorie")
        category_texts.append(category.text)
    
        # SCRAPPING OBJECT PURCHASE SECTION
    for i in range(1, 11):
        my_object = driver.find_element(by=By.ID,
                                        value=f"ctl0_CONTENU_PAGE_resultSearch_tableauResultSearch_ctl{i}_panelBlocObjet")
        object_texts.append(my_object.text)
        object_text2 = [s.replace("Objet :", "") for s in object_texts]
    
        # SCRAPPING PUBLIC PURCHASE SECTION
    for i in range(1, 11):
        my_public_purchaser = driver.find_element(by=By.ID,
                                                  value=f"ctl0_CONTENU_PAGE_resultSearch_tableauResultSearch_ctl{i}_panelBlocDenomination")
        public_purchase_texts.append(my_public_purchaser.text)
        public_purchase2 = [s.replace("Acheteur public :", "") for s in public_purchase_texts]
    
        # SCRAPPING THE END DATE SESSION
        datetime = driver.find_elements(by=By.CLASS_NAME,value="cloture-line")
    for dt in datetime:
        
        df_text = dt.text
        datetime_end_list.append(df_text)
        
    charToDelete = ''
    datetime_end_list2 = []
    
        
    for character in datetime_end_list:
        if character == charToDelete:
            continue
        datetime_end_list2.append(character)
         
        # SCRAPPING THE START DATE SESSION
    start_dt = driver.find_elements(by=By.CLASS_NAME,value="col-90")
    for dte in start_dt:
        dfe_text = dte.text
        start_dt_list.append(dfe_text)
        
   
    test = driver.find_element(by=By.ID, value='ctl0_CONTENU_PAGE_resultSearch_PagerBottom_ctl2')
    driver.execute_script("arguments[0].click();", test)
    time.sleep(5)

# CLEANING THE TEXTS

In [218]:
#CLEANING THE START DATE
start_dt_list_new = []
for i in range(len(start_dt_list)) :
    if start_dt_list[i].find('2022') != -1 :
        start_dt_list_new.append(start_dt_list[i])

In [219]:
len(start_dt_list_new)

30

In [220]:
start_dt_list_new1 = start_dt_list_new
for i in range(len(start_dt_list_new)) :
    start_dt_list_new1[i] = start_dt_list_new[i][-10:]

In [221]:
#INSTANTIATING THE PANDAS DATAFRAME
morocco_dataset = pd.DataFrame()

In [222]:
#creating dataframe for all list

features = {'Reference_id':ref_text2, 'Category':category_texts,'Object_Purchase':object_text2,
              'Public_Purchase':public_purchase2,'Location':city_texts, 'Start_date':start_dt_list_new1,
              'End_date':datetime_end_list2 }

#CONVERTING DICTIONARY TO DATAFRAME

data = pd.DataFrame(features)
morocco_dataset = morocco_dataset.append(data, ignore_index=True)

C:\Users\dell\AppData\Local\Temp\ipykernel_5792\2539719939.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  morocco_dataset = morocco_dataset.append(data, ignore_index=True)


In [223]:
# VIEWING THE FIRST FIVE OBSERVATIONS OF THE DATASET
morocco_dataset.head()

,Reference_id,Category,Object_Purchase,Public_Purchase,Location,Start_date,End_date
0,01/DSP/SAPT/2022,Travaux,APPEL A LA CONCURRENCE POUR LA GESTION DELEGU...,SOCIETE D'AMENAGEMENT POUR LA RECONVERSION DE...,Wilaya de Tanger Assilah,15/11/2022,17/04/2023\n11:00
1,1CGD2022,Services,Gestion déléguée du centre de traitement et d...,Commune d'OUJDA,OUJDA-ANGAD,16/12/2022,02/03/2023\n11:00
2,15/2022/CTSA/F,Fournitures,PRESTATIONS DE REALISATION DE LA 2ème PHASE D...,CASABLANCA TRANSPORT EN SITE AMENAGE SA,CASABLANCA,27/09/2022,28/02/2023\n10:00
3,07/2022/DRSMS,Fournitures,EQUIPEMENT DE L’HOPITAL DE PROXIMITE D’IMINTA...,"Directeur Régional de la Santé, région Marrak...",Wilaya de Marrakech Medina,16/12/2022,08/02/2023\n10:00
4,26139/B3/PIC,Fournitures,la Fourniture de Rails Neufs 60 E1 de 36 ml,OFFICE NATIONAL DES CHEMINS DE FER,KENITRA\nCASABLANCA\n...,15/12/2022,08/02/2023\n09:00


In [224]:
morocco_dataset.shape

(30, 7)

In [73]:
# CONVERTING THE DATASET TO A CSV FILE
morocco_dataset.to_csv('morocco_dataset.csv')

# GHANA 

# EXTRACTION SCRIPT FOR GHANA

In [227]:
contract_list = []

for i in range(1,90):
    link = f"https://www.ghaneps.gov.gh/epps/viewAllAwardedContracts.do?d-3998960-p={i}&selectedItem=viewAllAwardedContracts.do"
    page = requests.get(link)
    ghana = page.text
    soup = BeautifulSoup(ghana, "html.parser")
    
    rows = soup.find('table',{'id':'T01'}).find('tbody').find_all('tr')
    

    for row in rows:
        dic = {}
        dic['Tender_title'] = row.find_all('td')[0].getText(strip = True)
        dic['Procuring_entity'] = row.find_all('td')[1].getText(strip = True)
        dic['Supplier_name'] = row.find_all('td')[2].getText(strip = True)
        dic['Award_date'] = row.find_all('td')[3].getText(strip = True)
        dic['Award_amount'] = row.find_all('td')[4].getText(strip = True)
        
       
        contract_list.append(dic)
    

In [228]:
 #CONVERTING DICTIONARY INTO DATAFRAME
df2 = pd.DataFrame(contract_list)

In [230]:
#VIEWING THE HEAD OF THE DATASET
df2.head()

,Tender_title,Procuring_entity,Supplier_name,Award_date,Award_amount
0,Disposable Gloves,BEKWAI MUNICIPAL HOSPITAL,PEREGRINE FALCON COMPANY LIMITED,2022/12/14 15:10:55,0.60\r\n\t \t(GHS)
1,Disposable Gloves,BEKWAI MUNICIPAL HOSPITAL,PEREGRINE FALCON COMPANY LIMITED,2022/12/14 15:10:55,4.00\r\n\t \t(GHS)
2,Disposable Gloves,BEKWAI MUNICIPAL HOSPITAL,PEREGRINE FALCON COMPANY LIMITED,2022/12/14 15:10:55,5.00\r\n\t \t(GHS)
3,Disposable Gloves,BEKWAI MUNICIPAL HOSPITAL,PEREGRINE FALCON COMPANY LIMITED,2022/12/14 15:10:55,4.00\r\n\t \t(GHS)
4,Miscellaneous Consumables,BEKWAI MUNICIPAL HOSPITAL,A. BOAKYE KENDY ENTERPRISE,2022/12/14 16:49:38,7.18\r\n\t \t(GHS)


In [231]:
df2.shape

(20, 5)

In [53]:
#CONVERTING THE DATASET INTO CSV FILE
df.to_csv('ghana_dataset.csv', index = False)

# BENIN

# EXTRACTION SCRIPT FOR BENIN

In [233]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

chrome_driver_path = "C:\\Users\\dell\\Documents\\chrome driver\\chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.get("https://www.marches-publics.bj/appels-doffres?status=0")
time.sleep(10)

title_list = []
contracting_authority_list = []
delay_list = []
filing_list = []
pub_date_list = []

for i in range(34):
    
    title = driver.find_elements(by=By.CLASS_NAME, value="item-title")
    for i in title:
        each_title = i.text
        title_list.append(each_title)
    #print(title_list)

    
    contracting_authority = driver.find_elements(by=By.CLASS_NAME, value="regular")
    for auth in contracting_authority:
        each_auth = auth.text
        contracting_authority_list.append(each_auth)
    #print(contracting_authority_list)

    
    delay = driver.find_elements(by=By.CLASS_NAME, value="delay-section")
    for d in delay:
        each_delay = d.text
        delay_list.append(each_delay)
    #print(delay_list)

    
    filing = driver.find_elements(by=By.CLASS_NAME, value="limite-date-section")
    for dates in filing:
        each_date = dates.text
        filing_list.append(each_date)
    #print(filing_list)

    
    pub_date = driver.find_elements(by=By.CLASS_NAME, value="item-content")
    for date in pub_date:
        each_date = date.text
        pub_date_list.append(each_date)
    #print(pub_date_list)

    next_button = driver.find_element(by=By.CLASS_NAME, value="mat-paginator-navigation-next")
    driver.execute_script("arguments[0].click();", next_button)
    time.sleep(15)

C:\Users\dell\AppData\Local\Temp\ipykernel_5792\2606018493.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


# CLEANING THE TEXT

In [247]:
len(title_list) 
#del title_list[-30:]

In [248]:
len(contracting_authority_list)
#del contracting_authority_list[-30:]

In [166]:
#contracting_authority_list

In [249]:
len(filing_list)
#del filing_list[-30:]

In [171]:
#filing_list

In [238]:
filing_list_new = []
for i in range(len(filing_list)) :
    if filing_list[i].find(' à ') != -1 :
        filing_list_new.append(filing_list[i])

In [239]:
filing_list_new_red = []
for i in range(len(filing_list_new)):
    filing_list_new_red.append(filing_list_new[i][21:31])
filing_list_new_red

['03-12-2023',
 '18-01-2023',
 '11-01-2023',
 '06-01-2023',
 '05-01-2023',
 '05-01-2023',
 '05-01-2023',
 '04-01-2023',
 '03-01-2023',
 '29-12-2022',
 '29-12-2022',
 '29-12-2022',
 '29-12-2022',
 '28-12-2022',
 '28-12-2022',
 '28-12-2022',
 '28-12-2022',
 '27-12-2022',
 '26-12-2022',
 '26-12-2022',
 '22-12-2022',
 '22-12-2022',
 '21-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '19-12-2022',
 '19-12-2022',
 '19-12-2022',
 '19-12-2022',
 '19-12-2022',
 '16-12-2022',
 '16-12-2022',
 '16-12-2022',
 '15-12-2022',
 '14-12-2022',
 '14-12-2022',
 '14-12-2022',
 '14-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '12-12-2022',
 '12-12-2022',
 '12-12-2022',
 '09-12-2022',
 '08-12-2022',
 '08-12-2022',
 '08-12-2022',
 '08-12-2022',
 '08-12-2022',
 '07-12-2022']

In [256]:
len(filing_list_new_red)
del filing_list_new_red[-30:]

In [240]:
pub_date_list

['12-12-2022',
 '03-12-2024 à 10H30',
 'T_PAGEFCOM_55261',
 'SECRETARIAT PRMP MCVDD BUREAU 315',
 '15-12-2022',
 '18-01-2023 à 10H30',
 'T_DPET_62490',
 "Agence Nationale d'Approvisionnement en Eau Potable en Milieu Rural",
 '12-12-2022',
 '11-01-2023 à 10H30',
 'T_ST_56438',
 'Secrétariat de la PRMP',
 '05-12-2022',
 '06-01-2023 à 10H30',
 'S_DQSSE_66707',
 'Bureau111 du PAC',
 '05-12-2022',
 '05-01-2023 à 10H00',
 'T_DST_66375',
 'Secrétariat Permanent de la Personne Responsable des Marchés Publics, MAIRIE DE OUIDAH -B.P. 11 OUIDAH - TEL : (00229) 69 81 73 78, E.mail : spprmp.ouidah@gmail.com',
 '14-12-2022',
 '05-01-2023 à 10H30',
 'T_DST_66751',
 'Secrétaire permanent de la personne responsable des marchés publics de la Mairie de Cotonou',
 '28-11-2022',
 '05-01-2023 à 10H30',
 'S_DT_67214',
 'Bureau 111 DPRMP',
 '12-12-2022',
 '04-01-2023 à 10H30',
 'T_ST_62925',
 'MAIRIE DE SAVE',
 '12-12-2022',
 '03-01-2023 à 10H30',
 'F_ST_67438',
 'SP PRMP KETOU',
 '15-12-2022',
 '29-12-2022 à

In [241]:
Reference_id =[]
Start_date =[]
Offer_Sel_Date =[]
for i in range(0,(len(pub_date_list)-(30*4)),4) :
    Reference_id.append(pub_date_list[i+2])
    Start_date.append(pub_date_list[i])
    Offer_Sel_Date.append(pub_date_list[i+1])
for i in range(len(Offer_Sel_Date)):
    Offer_Sel_Date[i] = Offer_Sel_Date[i][:10]

In [242]:
Reference_id

['T_PAGEFCOM_55261',
 'T_DPET_62490',
 'T_ST_56438',
 'S_DQSSE_66707',
 'T_DST_66375',
 'T_DST_66751',
 'S_DT_67214',
 'T_ST_62925',
 'F_ST_67438',
 'T_ST_61211',
 'T_ST_52846',
 'T_ST_67162',
 'F_DG_57199',
 'F_CHUD-OP_55151',
 'F_CHUD-OP_67374',
 'T_DPAL_63917',
 'F_DDLP_56617',
 'F_ONAB_64382',
 'F_DSI_66636',
 'T_PAGEFCOM_55263',
 'PI_ST_58491',
 'T_RST_52158',
 'F_RAAF_52241',
 'T_DST_66376',
 'T_DG_57151',
 'S_DAF_66002',
 'F_DST_66637',
 'T_DGT_67142',
 'S_DIRECTION GENERALE_67579',
 'F_DT_58483']

In [243]:
Start_date

['12-12-2022',
 '15-12-2022',
 '12-12-2022',
 '05-12-2022',
 '05-12-2022',
 '14-12-2022',
 '28-11-2022',
 '12-12-2022',
 '12-12-2022',
 '15-12-2022',
 '09-12-2022',
 '06-12-2022',
 '30-11-2022',
 '07-12-2022',
 '07-12-2022',
 '06-12-2022',
 '13-12-2022',
 '07-12-2022',
 '25-11-2022',
 '05-12-2022',
 '12-12-2022',
 '14-12-2022',
 '14-12-2022',
 '29-11-2022',
 '05-12-2022',
 '29-11-2022',
 '25-11-2022',
 '18-11-2022',
 '28-11-2022',
 '18-11-2022']

In [244]:
Offer_Sel_Date

['03-12-2024',
 '18-01-2023',
 '11-01-2023',
 '06-01-2023',
 '05-01-2023',
 '05-01-2023',
 '05-01-2023',
 '04-01-2023',
 '03-01-2023',
 '29-12-2022',
 '29-12-2022',
 '29-12-2022',
 '29-12-2022',
 '28-12-2022',
 '28-12-2022',
 '28-12-2022',
 '28-12-2022',
 '27-12-2022',
 '26-12-2022',
 '26-12-2022',
 '22-12-2022',
 '22-12-2022',
 '21-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '19-12-2022']

In [257]:
lenn = [len(Reference_id),len(Start_date), len(Offer_Sel_Date), len(contracting_authority_list), len(filing_list_new_red), len(title_list)]
lenn

[30, 30, 30, 30, 30, 30]

['03-12-2023',
 '18-01-2023',
 '11-01-2023',
 '06-01-2023',
 '05-01-2023',
 '05-01-2023',
 '05-01-2023',
 '04-01-2023',
 '03-01-2023',
 '29-12-2022',
 '29-12-2022',
 '29-12-2022',
 '29-12-2022',
 '28-12-2022',
 '28-12-2022',
 '28-12-2022',
 '28-12-2022',
 '27-12-2022',
 '26-12-2022',
 '26-12-2022',
 '22-12-2022',
 '22-12-2022',
 '21-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '20-12-2022',
 '19-12-2022',
 '19-12-2022',
 '19-12-2022',
 '19-12-2022',
 '19-12-2022',
 '16-12-2022',
 '16-12-2022',
 '16-12-2022',
 '15-12-2022',
 '14-12-2022',
 '14-12-2022',
 '14-12-2022',
 '14-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '13-12-2022',
 '12-12-2022',
 '12-12-2022',
 '12-12-2022',
 '09-12-2022',
 '08-12-2022',
 '08-12-2022',
 '08-12-2022',
 '08-12-2022',
 '08-12-2022',
 '07-12-2022']

In [258]:
benin_dataset = pd.DataFrame()
#creating dataframe for all list
features = {'Reference_id':Reference_id, 'Contracting_Authority':contracting_authority_list,'Offer_Sel_Date':Offer_Sel_Date,
              'Title_entry':title_list,'Start_date':Start_date,
              'End_date':filing_list_new_red}
#CREATING DATAFRAME FOR BENIN DATASET
data = pd.DataFrame(features)
benin_dataset = benin_dataset.append(data, ignore_index=True)

C:\Users\dell\AppData\Local\Temp\ipykernel_5792\918480642.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  benin_dataset = benin_dataset.append(data, ignore_index=True)


In [259]:
#VIEWING THE HEAD OF THE DATASET
benin_dataset.head(10)

,Reference_id,Contracting_Authority,Offer_Sel_Date,Title_entry,Start_date,End_date
0,T_PAGEFCOM_55261,Ministère du Cadre de Vie et du Développement ...,03-12-2024,Travaux de mise en œuvre des plans d'aménageme...,12-12-2022,03-12-2023
1,T_DPET_62490,Agence Nationale d'Approvisionnement en Eau Po...,18-01-2023,Réalisation des travaux des Systèmes d'Approvi...,15-12-2022,18-01-2023
2,T_ST_56438,Commune d'Athiémé,11-01-2023,Travaux de réhabilitation de l’ancien ouvrage ...,12-12-2022,11-01-2023
3,S_DQSSE_66707,Port Autonome de Cotonou,06-01-2023,Accord cadre pour le regroupement et le convoy...,05-12-2022,06-01-2023
4,T_DST_66375,Commune de Ouidah,05-01-2023,Travaux de réalisation d'une clôture partielle...,05-12-2022,05-01-2023
5,T_DST_66751,Mairie de Cotonou,05-01-2023,Construction et équipements de bâtiments de ty...,14-12-2022,05-01-2023
6,S_DT_67214,Port Autonome de Cotonou,05-01-2023,Sondage bathymétrique du bassin et chenal d’ac...,28-11-2022,05-01-2023
7,T_ST_62925,Commune de Savè,04-01-2023,Travaux d’aménagement de la rocade de OUOGHI a...,12-12-2022,04-01-2023
8,F_ST_67438,Commune de Kétou,03-01-2023,Acquisition de pièce des rechange dans le cadr...,12-12-2022,03-01-2023
9,T_ST_61211,Commune d'Adjohoun,29-12-2022,Travaux de réhabilitation des bureaux de la Ré...,15-12-2022,29-12-2022


In [215]:
#CONVERTING THE DATASET TO CSV FILE
benin_dataset.to_csv('benin_dataset.csv', index = False)